# Text Generation with RNNs
https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/

https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

### 1 Dataset
Define the path of the file, you want to read and train the model on


In [2]:
'''TODO: set the path of the file'''
path_to_file = 'Shakespeare_karpathy.txt'
text = open(path_to_file, encoding='utf-8').read()


#### Inspect the dataset
Take a look at the first 250 characters in text

In [3]:
print(text[:250])

That, poor contempt, or claim'd thou slept so faithful,
I may contrive our father; and, in their defeated queen,
Her flesh broke me and puttance of expedition house,
And in that same that ever I lament this stomach,
And he, nor Butly and my fury, kno


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

62 unique characters


### 2 Process the dataset for the learning task
The task that we want our model to achieve is: given a character, or a sequence of characters, what is the most probable next character?

To achieve this, we will input a sequence of characters to the model, and train the model to predict the output, that is, the following character at each time step. RNNs maintain an internal state that depends on previously seen elements, so information about all characters seen up until a given moment will be taken into account in generating the prediction.

#### Vectorize the text
Before we begin training our RNN model, we'll need to create a numerical representation of our text-based dataset. To do this, we'll generate two lookup tables: one that maps characters to numbers, and a second that maps numbers back to characters. Recall that we just identified the unique characters present in the text.

In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
text_as_int = np.array([char2idx[c] for c in text])

# Create a mapping from indices to characters
idx2char = np.array(vocab)


This gives us an integer representation for each character. Observe that the unique characters (i.e., our vocabulary) in the text are mapped as indices from 0 to len(unique). Let's take a peek at this numerical representation of our dataset:

In [6]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  ':' :   7,
  ';' :   8,
  '?' :   9,
  'A' :  10,
  'B' :  11,
  'C' :  12,
  'D' :  13,
  'E' :  14,
  'F' :  15,
  'G' :  16,
  'H' :  17,
  'I' :  18,
  'J' :  19,
  ...
}




We can also look at how the first part of the text is mapped to an integer representation:

In [7]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'That, poor co' ---- characters mapped to int ---- > [29 43 36 55  4  1 51 50 50 53  1 38 50]


#### Defining a method to encode one hot labels

In [8]:
def one_hot_encode(arr, n_labels):
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot


#### Defining a method to make mini-batches for training

In [9]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr) // batch_size_total
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n + seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n + seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y


## 3 The Recurrent Neural Network (RNN) model


###### Check if GPU is available

In [10]:
train_on_gpu = torch.cuda.is_available()
print ('Training on GPU' if train_on_gpu else 'Training on CPU')

Training on GPU



### Declaring the model

In [11]:
class VanillaCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        self.rnn = nn.RNN(len(self.vocab), self.n_hidden, self.n_layers, batch_first=True, dropout=self.drop_prob)   #batch_first=True -> input and output tensors are provided as (batch, seq, feature)
        self.fc = nn.Linear(self.n_hidden, len(self.vocab))

    def forward(self, x, hidden):

        # Passing in the input and hidden state into the model and obtaining outputs
        # x of shape (seq_len, batch, input_size)
        # hidden of shape (num_layers * num_directions, batch, hidden_size)
        out, hidden_t = self.rnn(x, hidden)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        # return the final output and the hidden state
        return out, hidden_t

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        hidden = torch.zeros(self.n_layers, batch_size, self.n_hidden)

        return hidden

In [12]:
class LSTMCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        self.lstm = nn.LSTM(len(self.vocab), self.n_hidden, self.n_layers, dropout=self.drop_prob, batch_first=True)
        self.fc = nn.Linear(self.n_hidden, len(self.vocab))


    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        # return the final output and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        hidden_state = torch.zeros(self.n_layers, batch_size, self.n_hidden).cuda()
        cell_state = torch.zeros(self.n_layers, batch_size, self.n_hidden).cuda()
        # cell state should be included as for now the model only backpropagates the loss over the hidden states and not the cell states
        return (hidden_state, cell_state)



#### Declaring the train method


train(vanilla_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

In [13]:
def train(model, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        model: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    model.train()

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if (train_on_gpu):
        model.cuda()

    counter = 0
    n_vocab = len(model.vocab)
    for e in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size)
        
        '''TODO: use the get_batches function to generate sequences of the desired size'''
        dataset = get_batches(data, batch_size, seq_length)

        for x, y in dataset:
            counter += 1
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_vocab)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if (train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            if type(h) is tuple:
                if train_on_gpu:
                    h = tuple([each.data.cuda() for each in h])
                else:
                    h = tuple([each.data for each in h])
            else:
                if train_on_gpu:
                    h = h.data.cuda()
                else:
                    h = h.data
            # zero accumulated gradients
            model.zero_grad()
            output, h = model(inputs, h) 
            loss = criterion(output, targets.view(-1).long())
            
            # perform backprop
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = model.init_hidden(batch_size)
                val_losses = []
                model.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_vocab)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    if type(val_h) is tuple:
                        if train_on_gpu:
                            val_h = tuple([each.data.cuda() for each in val_h])
                        else:
                            val_h = tuple([each.data for each in val_h])
                    else:
                        if train_on_gpu:
                            val_h = val_h.data.cuda()
                        else:
                            val_h = val_h.data

                    inputs, targets = x, y
                    if (train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    output, val_h = model(inputs, val_h)

                    val_loss = criterion(output, targets.view(-1).long())

                    val_losses.append(val_loss.item())

                print("Epoch: {}/{}...".format(e + 1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
                input_eval = 'Dear'
                print(sample(model, 1000, prime=input_eval, top_k=10))
                
                model.train()  # reset to train mode after iterationg through validation data


##### Defining a method to generate the next character

In [14]:
def predict(model, char, h=None, top_k=None):
    ''' 
    Given a character, predict the next character.
    Returns the predicted character and the hidden state.
    '''

    # tensor inputs
    x = np.array([[char2idx[char]]])
    x = one_hot_encode(x, len(model.vocab))
    inputs = torch.from_numpy(x)
    
    if (train_on_gpu):
        inputs = inputs.cuda()

    # detach hidden state from history
    if type(h) is tuple:
        if train_on_gpu:
            h = tuple([each.data.cuda() for each in h])
        else:
            h = tuple([each.data for each in h])
    else:
        if train_on_gpu:
            h = h.data.cuda()
        else:
            h = h.data
    output, h = model(inputs, h)

    # get the character probabilities
    p = F.softmax(output, dim=1).data
    if (train_on_gpu):
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(model.vocab))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return idx2char[char], h



#### Declaring a method to generate new text

In [15]:
def sample(model, size, prime='The', top_k=None):
    if (train_on_gpu):
        model.cuda()
    else:
        model.cpu()

    model.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    for ch in prime:
        char, h = predict(model, ch, h, top_k=top_k)

    chars.append(char)

    for ii in range(size):
        char, h = predict(model, char, h, top_k=top_k)
        chars.append(char)

    model.train()
    return ''.join(chars)


#### Generate new Text using the RNN model

###### Define and print the net

In [31]:
''''TODO: Try changing the number of units in the network to see how it affects performance'''
n_hidden = 256
n_layers = 2

vanilla_model = VanillaCharRNN(vocab, n_hidden, n_layers)
print(vanilla_model)
lstm_model = LSTMCharRNN(vocab, n_hidden, n_layers)
print(lstm_model)

VanillaCharRNN(
  (rnn): RNN(62, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=62, bias=True)
)
LSTMCharRNN(
  (lstm): LSTM(62, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=62, bias=True)
)



###### Declaring the hyperparameters

In [73]:
''''TODO: Try changing the hyperparameters in the network to see how it affects performance'''
batch_size = 50
seq_length = 50
n_epochs = 20  # start smaller if you are just testing initial behavior


##### Train the model and have fun with the generated texts

In [74]:
train(vanilla_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

Epoch: 2/20... Step: 50... Loss: 1.4080... Val Loss: 1.7587
Dear spaence thingion.

MARK ANTONY:
O had it, and there is betument of thy mather of suffench another
Head my ground and belick'd:
But well in a ganmon so be oartes.

HELENA:
Alas, so stor blind the way blunt and heart,
And that talks, as honour would,
See so brother's a frown-by your guilous whife will put you know when the dutch your crafain
A hands are passage, his amoul from our mander's tought me.

DUKE OF DOUGLAS:
That the way
Shall now all thee atter the part; but you we have sin and thee take with a speekings
When I may ame.

Second Lord:

MALVOLIO:
A part,
Then would not give another te's full,
But your cried,
And blother:
Let I have thy say, and first which his baur you a speak my braves arm the gud my be sure;
And to hast have that that thy very mysee so,
For them; and there's good storn and since
To day,
Good dead
To make your come the hands. Let thee to die fire offent shall do buces and alsow all theat his stain

Epoch: 13/20... Step: 450... Loss: 1.3008... Val Loss: 1.7561
Dear them fon will fall my house; she do not should be prigion,
And man were you words may be prick that is the world can change to be treature;
Tirn the crom they arge
For care trepering out,
Murture's mecrion:
Let us all this present face down that desmed, I have bloody of thee, I will not;
First. Brutus that every to all the world un their pity, and I would never days: I thought, and throwers,
Plecks; it, as men flint all.
Himself it nature, when I have done had them to a spirit
Wat would said
I shall fellour flonces.

DONTA:
Ay, to life fors to stopperous some it was shake him, have stay'd are provise in tert us wit, my flad it satiog, whose take thee in the gadst
As tongheling of the body and great should not gain dim as have I mept to day, I have a mone melanched to you may gages:
Never feerly done thy heart,
Wro sains's heaven for I'll send of men as thine and good people; my let to the cause out
And speet with their 

In [75]:
train(lstm_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

Epoch: 2/20... Step: 50... Loss: 0.7554... Val Loss: 2.2096
Dear:
No, I opher; uncle my lord,
And hang make yet face; and in the enemy,
When he in a gooding some goodness I tole;
Then thy I am breath.
Though her dearly courtest towers, and single
Wills dir to his oaths over desire,
Who make never I scave a layful place.
Here comes theme upon the world?

DUKE OF YORK:
More than are port in this tame.

MOLOF:
And that it s?

THENLAWIUS:

SPEED:
Why, my lord.

TLOUCESTER:
They are, undour me, and make you shall difflaction.

SARIIGUS:
Here'd madam; I will. O, spay I can meen a copjain.

RODERIGUS:
That'fole, seem.

DUCHESS OF YORK:
I saw them through their parts of love: contempt,
To give my son, you doth uncone that it sweet me by thee
That I should not trat thee spirits
And stat's erecuses the like and by that
blowss and melly again, or my time:
Lrand to have you all or his brother.

CLEOPATRA:
No, sinm Ansexarde's judgment in
leg outsax labour happier with you: but I'll speak
But every

Epoch: 13/20... Step: 450... Loss: 0.5537... Val Loss: 2.3316
Dear:
Ay, sir, came command this pearact, who keng it perron.

Second Servingman:
By these twongher-mand, with my wearth
To great, sir, as if entreat basion! guilty, if all the opposition and tell us you know,
That in the gods behomit of any boar and gid.

MALVOLIO:
Nay, cun ement the way nore.

TROILUS:
I cannat to people; who hep crow Herrible princes,
As let me beguiled them, in he's in the gone.

LLONCESTER:
I will not way whiphes; but who knows no peacle.

MALVOLIO:
Nay, twise e must donouge, let him by heaven's and fife, with his amazed: he remember,
She sent great are down beacely here; and, no more than are,
I'll struck from every kingdom, had wor ever
straight take it: and given
Go him so impudent. If it not fair heast that high hands,
That it were thou wouldst bear. I had not more daughter gove
The music convening till be her. but your majesty.

DUKE OF YORK:
I'll prove beauty head; I think you, sir;
I to have my h

##### Generate text

In [76]:
print(sample(vanilla_model, 500, prime='The', top_k=10))

Therefore them that fellow
As I lose a woman sooner of the lead we are exarge,
To stay a wissius some so shipler' plames at common friends, my boinmes the payf is a chinter will emtunce, and farch at shall pood leave performs, and with their futy, good common.
Sir, and they he love to their same clance of our read;
I think it actruar of this body so brow of this rather: how have thee all oursown.

BASSANIO:
Had when should I dight
Speat for you, but where I will: that is an my swown to you.

GLOUCES


In [77]:
print(sample(lstm_model, 500, prime='The', top_k=10))

There's
more her deliver your chains:
I am Cleapor, and her wence andich
And dore is moved some broth
So umparty dow my father, one it enemy that it as me
More than your heart, which now does,
Agest in the kingdom.

LOONSAMONANO:
O, I will not constience?

LAUNTHAS:
OF, it is with the truth, he may contein;
Hent in this fortune with consucien of the line:
That sons was not hope I gentle of the clame.
Here is my good at the vlither
With so swear it may well enemy of their hands,
That thy your wife, t


## Results:
- Changing the number of layers: 5 layers just produced nonsense text, maybe the higher number of layers would need significantly higher number of training epochs. 2 layers seem to be fine with 20 epochs and produce almost readable text.
- Changing the sequence length: 20 or 50 does not seem to be different, produces longer texts and fewer dialogue, 100 produces much more dialogue in most samples 
- Changing the batch size: 10 seems to work fine, 30 produces more random signs in words, 50 looks really good, 100 too
- 2 layers, batchsize 50 and sequence length 50 was, with the empirically chosen values, the best result